In [5]:
import pandas as pd
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
import numpy as np

In [6]:
p9 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2009.csv', sep=',', low_memory=False)

In [7]:
p10 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2010.csv', sep=',', low_memory=False)

In [8]:
p11 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2011.csv', sep=',', low_memory=False)

In [9]:
p12 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2012.csv', sep=',', low_memory=False)

In [10]:
p13 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2013.csv', sep=',', low_memory=False)

In [11]:
p14 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2014.csv', sep=',', low_memory=False)

In [12]:
p15 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2015.csv', sep=',', low_memory=False)

In [18]:
mylist = []

for chunk in pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2016.csv', sep=',', chunksize=20000):
    mylist.append(chunk)
p16 = pd.concat(mylist, axis = 0)


In [19]:
p17 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2017.csv', sep=',', low_memory=False)

In [23]:
mylist = []
for chunk in pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2018.csv', sep=',', chunksize=20000,low_memory=False):
    mylist.append(chunk)
p18 = pd.concat(mylist, axis = 0)
del mylist

In [24]:
passergroup18 = p18.groupby(['posteam', 'passer_player_name'])["pass_touchdown", "yards_gained"].apply(lambda x : x.astype(int).sum())
passergroup18.head()

pass_touchdown  yards_gained
posteam passer_player_name                              
ARI     J.Rosen                         11          1960
        L.Fitzgerald                     1            32
        M.Glennon                        1           166
        S.Bradford                       2           367
ATL     M.Ryan                          35          4632

In [29]:
#tackles
#tacklescoring - https://imgur.com/Ig0eCsh
#tacklepositions - LB, S, QB, RB, WR, TE, PK, PN
#groupby players and their tackles, create column to determine points from tackles, join with roster data, filter by eligible positions, 
tacklepts18one = pd.DataFrame({'tackles' : p18.groupby( [ "assist_tackle_1_player_id"] ).size()}).reset_index()
tacklepts18two = pd.DataFrame({'tackles' : p18.groupby( [ "assist_tackle_2_player_id"] ).size()}).reset_index()
type(tacklepts18one)

pandas.core.frame.DataFrame

In [30]:
tacklepts18one.head()

,assist_tackle_1_player_id,tackles
0,00-0021140,7
1,00-0022161,5
2,00-0022247,14
3,00-0023173,2
4,00-0023259,13


In [31]:
#gotta figure out how to merge them, something about the column names not being indexed properly
tacklepts18 = pd.merge(tacklepts18one, tacklepts18two, left_on = 'assist_tackle_1_player_id', right_on = 'assist_tackle_2_player_id', how='inner')

In [32]:
tacklepts18.head()

,assist_tackle_1_player_id,tackles_x,assist_tackle_2_player_id,tackles_y
0,00-0021140,7,00-0021140,4
1,00-0022161,5,00-0022161,6
2,00-0022247,14,00-0022247,19
3,00-0023173,2,00-0023173,1
4,00-0023259,13,00-0023259,13


In [33]:
#Add both groups of tackle assists together, multiply by 0.5 for fantasy point total
tacklepts18['tackles_xy'] = (tacklepts18['tackles_x'] + tacklepts18['tackles_y']) * 0.5
tacklepts18.head()

,assist_tackle_1_player_id,tackles_x,assist_tackle_2_player_id,tackles_y,tackles_xy
0,00-0021140,7,00-0021140,4,5.5
1,00-0022161,5,00-0022161,6,5.5
2,00-0022247,14,00-0022247,19,16.5
3,00-0023173,2,00-0023173,1,1.5
4,00-0023259,13,00-0023259,13,13.0


In [34]:
rosters18 = pd.read_csv(r'nflscrapR-data\roster_data\regular_season\reg_roster_2018.csv', sep = ',', low_memory = False )

In [35]:
#Add the column of  tackle assist points to the roster data
#Get rid of extra player id column
rosters18 = pd.merge(rosters18,tacklepts18[['tackles_xy','assist_tackle_1_player_id']],left_on='gsis_id',right_on='assist_tackle_1_player_id')
rosters18.head()


,season,season_type,full_player_name,abbr_player_name,team,position,gsis_id,tackles_xy,assist_tackle_1_player_id
0,2018,reg,Anthony Sherman,A.Sherman,KC,RB,00-0028074,1.0,00-0028074
1,2018,reg,C.J. Ham,C.Ham,MIN,FB,00-0032918,2.0,00-0032918
2,2018,reg,Charone Peake,C.Peake,NYJ,WR,00-0032959,1.0,00-0032959
3,2018,reg,Darius Jennings,D.Jennings,TEN,WR,00-0031972,1.0,00-0031972
4,2018,reg,De'Lance Turner,D.Turner,BAL,RB,00-0034201,1.0,00-0034201


In [36]:
rosters18.columns= ['season', 'season_type', 'full_player_name', 'abbr_player_name', 'team',
       'position', 'gsis_id', 'tackle_assist_points', 'assist_tackle_1_player_id']
rosters18.drop(['assist_tackle_1_player_id'], axis=1,inplace=True)
rosters18.head()

,season,season_type,full_player_name,abbr_player_name,team,position,gsis_id,tackle_assist_points
0,2018,reg,Anthony Sherman,A.Sherman,KC,RB,00-0028074,1.0
1,2018,reg,C.J. Ham,C.Ham,MIN,FB,00-0032918,2.0
2,2018,reg,Charone Peake,C.Peake,NYJ,WR,00-0032959,1.0
3,2018,reg,Darius Jennings,D.Jennings,TEN,WR,00-0031972,1.0
4,2018,reg,De'Lance Turner,D.Turner,BAL,RB,00-0034201,1.0


In [37]:
print(rosters18.size)

176


In [38]:
rosters18.sort_values('tackle_assist_points',ascending=False)
print(rosters18[rosters18['position']=='LB'])

Empty DataFrame
Columns: [season, season_type, full_player_name, abbr_player_name, team, position, gsis_id, tackle_assist_points]
Index: []


In [39]:
#gotta figure out how to merge them, something about the column names not being indexed properly
tacklepts18 = pd.merge(tacklepts18one, tacklepts18two, left_on = 'assist_tackle_1_player_id', right_on = 'assist_tackle_2_player_id', how='inner')

In [40]:
tacklepts18.head()

,assist_tackle_1_player_id,tackles_x,assist_tackle_2_player_id,tackles_y
0,00-0021140,7,00-0021140,4
1,00-0022161,5,00-0022161,6
2,00-0022247,14,00-0022247,19
3,00-0023173,2,00-0023173,1
4,00-0023259,13,00-0023259,13


In [41]:
#Add both groups of tackle assists together, multiply by 0.5 for fantasy point total
tacklepts18['tackles_xy'] = (tacklepts18['tackles_x'] + tacklepts18['tackles_y']) * 0.5
tacklepts18.head()

,assist_tackle_1_player_id,tackles_x,assist_tackle_2_player_id,tackles_y,tackles_xy
0,00-0021140,7,00-0021140,4,5.5
1,00-0022161,5,00-0022161,6,5.5
2,00-0022247,14,00-0022247,19,16.5
3,00-0023173,2,00-0023173,1,1.5
4,00-0023259,13,00-0023259,13,13.0


In [42]:
rosters18 = pd.read_csv(r'nflscrapR-data\roster_data\regular_season\reg_roster_2018.csv', sep = ',', low_memory = False )

In [43]:
#Add the column of  tackle assist points to the roster data
#Get rid of extra player id column
rosters18 = pd.merge(rosters18,tacklepts18[['tackles_xy','assist_tackle_1_player_id']],left_on='gsis_id',right_on='assist_tackle_1_player_id')
rosters18.head()


,season,season_type,full_player_name,abbr_player_name,team,position,gsis_id,tackles_xy,assist_tackle_1_player_id
0,2018,reg,Anthony Sherman,A.Sherman,KC,RB,00-0028074,1.0,00-0028074
1,2018,reg,C.J. Ham,C.Ham,MIN,FB,00-0032918,2.0,00-0032918
2,2018,reg,Charone Peake,C.Peake,NYJ,WR,00-0032959,1.0,00-0032959
3,2018,reg,Darius Jennings,D.Jennings,TEN,WR,00-0031972,1.0,00-0031972
4,2018,reg,De'Lance Turner,D.Turner,BAL,RB,00-0034201,1.0,00-0034201


In [44]:
rosters18.columns= ['season', 'season_type', 'full_player_name', 'abbr_player_name', 'team',
       'position', 'gsis_id', 'tackle_assist_points', 'assist_tackle_1_player_id']
rosters18.drop(['assist_tackle_1_player_id'], axis=1,inplace=True)
rosters18.head()

,season,season_type,full_player_name,abbr_player_name,team,position,gsis_id,tackle_assist_points
0,2018,reg,Anthony Sherman,A.Sherman,KC,RB,00-0028074,1.0
1,2018,reg,C.J. Ham,C.Ham,MIN,FB,00-0032918,2.0
2,2018,reg,Charone Peake,C.Peake,NYJ,WR,00-0032959,1.0
3,2018,reg,Darius Jennings,D.Jennings,TEN,WR,00-0031972,1.0
4,2018,reg,De'Lance Turner,D.Turner,BAL,RB,00-0034201,1.0


In [45]:
print(rosters18.size)

176


In [46]:
rosters18.sort_values('tackle_assist_points',ascending=False)
print(rosters18[rosters18['position']=='LB'])

Empty DataFrame
Columns: [season, season_type, full_player_name, abbr_player_name, team, position, gsis_id, tackle_assist_points]
Index: []
